###

In [1]:
import torch
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
from torch import nn
import os
from datetime import datetime
import time
import random
import cv2
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm
import re, magic

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from  torch.cuda.amp import autocast, GradScaler

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
import cv2
#import pydicom
import timm #from efficientnet_pytorch import EfficientNet
from scipy.ndimage import zoom
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix

#vit
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

In [ ]:
import torch.distributed as dist
import torch.multiprocessing as mp
#from torch.nn.parallel import DistributedDataParallel as DDP
from apex.parallel import DistributedDataParallel as DDP

os.environ["CUDA_VISIBLE_DEVICES"] = '0, 1' #본인이 사용하고 싶은 GPU 넘버를 써주면 됨
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '53097'         # 좀 큰 숫자로 맞추면 됨 작은 숫자는 에러발생!

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# init!
torch.distributed.init_process_group(backend='nccl', init_method="env://", rank =0, world_size=1)  # rank should be 0 ~ world_size-1


In [3]:
CFG = {
    'fold_num': 5,
    'seed': 0,
    'model': 'tf_efficientnet_b3_ns',
    'img_size': 300,
    'epochs': 50,
    'train_bs': 16,
    'valid_bs': 16,
    'T_0': 10,
    'lr': 1e-4,
    'min_lr': 1e-6,
    'weight_decay':1e-5,
    'num_workers': 0,
    'accum_iter': 2, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda:0'
}

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

In [5]:
class ColonDataset(Dataset):
    def __init__(
        self, df, data_root, transforms=None):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.df.iloc[index]['label']
          
        path = "{}/{}".format(self.data_root, self.df.iloc[index]['image_id'])
        #path = "{}/{}".format(self.data_root[index], self.df.iloc[index]['image_id'])
        img  = get_img(path)
        
        if self.transforms:
            img = self.transforms(image=img)['image']
            
        return img

In [6]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize, SmallestMaxSize
)

from albumentations.pytorch import ToTensorV2

def get_inference_transforms():
    return Compose([
            CenterCrop(always_apply=True,p=1.0, height=CFG['img_size'], width=CFG['img_size']),
            RandomResizedCrop(
            height=CFG['img_size'], 
            width=CFG['img_size'], 
            scale=(0.60, 0.64),
            ratio=(0.90, 1.10),
            always_apply=True
            ),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
#            VerticalFlip(p=0.5),
#             HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
#             RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

In [7]:
class ColonImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class=2, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained, num_classes=n_class)
        try :
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, n_class)

        except :
            
            try : 
                n_features = self.model.fc.in_features
                self.model.fc = nn.Linear(n_features, n_class)
            
            except :
                try:
                    n_features = self.model.head.fc.in_channels
                    self.model.head.fc = nn.Conv2d(n_features,n_class,kernel_size=(1, 1), stride=(1, 1))
            
                except:
                    n_features = self.model.head.in_features
                    self.model.head = nn.Linear(n_features, n_class)
            
    def forward(self, x):
        x = self.model(x)
        return x

In [8]:
def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        
        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
        print(image_preds_all)
    
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    print(f'final: {image_preds_all}')
    return image_preds_all

In [3]:
test_dir = 'test_data/'
#test_dir = test.dir.values
model_dir = 'tf_efficientnet_b3_ns/tf_efficientnet_b3_ns_38'

if __name__ == '__main__':
     # for training only, need nightly build pytorch

    seed_everything(CFG['seed'])

    #test = []
    test_ds = ColonDataset(test, test_dir, transforms=get_inference_transforms())


    tst_loader = torch.utils.data.DataLoader(
        test_ds, 
        batch_size=CFG['valid_bs'],
        num_workers=CFG['num_workers'],
        shuffle=False,
        pin_memory=True,
    )

    device = torch.device(CFG['device'])
    model = ColonImgClassifier(CFG['model'], 134).to(device)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    
    model.to(device)
    model = DDP(model)
    
    tst_preds = []

    model.load_state_dict(torch.load(model_dir))

    with torch.no_grad():
            tst_preds += [inference_one_epoch(model, tst_loader, device)]


    #tst_preds = inference_one_epoch(model, tst_loader, device)
    tst_preds = np.mean(tst_preds, axis=0) 
    print(f'mean of tst_preds = {tst_preds}')
    #print('test loss = {:.5f}'.format(log_loss(valid_.label.values, val_preds)))
    #print('test accuracy = {:.5f}'.format((valid_.label.values==np.argmax(val_preds, axis=1)).mean()))

    #  del model, tst_loader
    # torch.cuda.empty_cache()
        


NameError: name 'seed_everything' is not defined

NameError: name 'tst_preds' is not defined

In [132]:
# test
# test['label'] = test['image_id'].apply(lambda x : 0 if 'cancer' in x else 1)
test['pred'] = np.argmax(tst_preds, axis=1)
#test['pred'] = tst_preds
test['confidence score'] =np.max(tst_preds, axis=1)
#test_acc = np.sum(test.label == test.pred) / len(test)
#test_matrix = confusion_matrix(test['label'], test['pred'])
#print(round(test_acc, 5))
#print(test_matrix)